# DBQA GCI Circuit Synthesis

Variational quantum eigensolver (VQE) is commonly used for ground state preparation. In this notebook, we demonstrate how to use a few steps of double-bracket quantum algorithms (DBQAs) to enhance the performance of VQE.

To achieve this, we use the `qibo` package [1] to first train a warm-start VQE circuit, and from there, run a few steps of DBQAs [2] realized by group commutator iterations (GCIs) [3].

Then, the final circuit may be converted into `qiskit` or `pytekt` compatible formats for running on emulators or quantum hardwares.

[1] https://github.com/qiboteam/qibo

[2] M. Gluza, “Double-bracket quantum algorithms for diagonalization,” Quantum, vol. 8, p. 1316, Apr. 2024, doi: 10.22331/q-2024-04-09-1316. https://arxiv.org/abs/2206.11772 

[3] M. Robbiati et al., “Double-bracket quantum algorithms for high-fidelity ground state preparation,” Aug. 07, 2024, arXiv: arXiv:2408.03987. doi: 10.48550/arXiv.2408.03987.https://arxiv.org/abs/2408.03987

In this example notebook, we perform numerical simulations of DBQA for the XXZ model with a periodic boundary condition, whose hamiltonian is given by:

$$
H_0 = \sum _{k=0}^L \left( X_{k} X_{k + 1} + Y_{k} Y_{k + 1} + \delta Z_{k}Z_{k + 1} \right)
$$

In [ ]:
import qibo
from qibo.backends import construct_backend
from qibo import hamiltonians
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
qibo.set_backend("tensorflow")
vqe_backend = construct_backend(backend="tensorflow")

In [ ]:
nqubits = 4
delta = 0.5
ham = hamiltonians.XXZ(nqubits, delta, dense=False)
plt.imshow(np.real(ham.matrix))
plt.colorbar()

## Step 1: train VQE (optional)
We define the loss function to be the energy expectation $\langle H\rangle$.
The following cells contain the complete process of VQE training using Qibo.
While the user can modify the code and train their VQE, this section is optional if you wish to load from existing results, for which, proceed to step 2.

In [ ]:
from boostvqe import ansatze
from copy import deepcopy
from functools import partial
from boostvqe.training_utils import vqe_loss
from boostvqe.utils import *

In [ ]:
# helper functions
def exact_expectation_boost(ham, circ):
    # calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
        ham.backend.execute_circuit(circuit=circ).state())

In [ ]:
# build ansatz circuit
nlayer = 1
ansatz_circ = ansatze.hdw_efficient(nqubits, nlayer)
print(ansatz_circ.draw())

In [ ]:
# build zero state
zero_state = ham.backend.zero_state(nqubits)
# initial params
params_len = len(ansatz_circ.get_parameters())
# fix numpy seed to ensure replicability of the experiment
seed = 10
np.random.seed(seed)
initial_params = np.random.uniform(-np.pi, np.pi, params_len)
print('Initial parameters:', initial_params)
# initial energy
c0 = deepcopy(ansatz_circ)
c0.set_parameters(initial_params)
target_energy = np.real(np.min(np.asarray(ham.eigenvalues())))
print('Target enegry:', target_energy)
print('Initial energy:', exact_expectation_boost(ham, c0).numpy())
print('Net difference:', exact_expectation_boost(ham, c0).numpy()-target_energy)

In [ ]:
# define the qibo loss function
objective_boost = partial(vqe_loss)
# logging hisotry
params_history, loss_history, grads_history, fluctuations = [], [], [], []
# set optimizer
optimizer = 'sgd'
maxiter = 1500
nmessage = 500
learning_rate = 5e-2
tol = 1e-5 # for `sgd` this is required but not used
params_history.append(initial_params)

### Train VQE

In [ ]:
param = params_history[-1]
(
    partial_results,
    partial_params_history,
    partial_loss_history,
    partial_grads_history,
    partial_fluctuations,
    vqe,
) = train_vqe(
    deepcopy(ansatz_circ),
    ham,  # Fixed hamiltonian
    optimizer,
    param,
    tol=tol,
    niterations=maxiter, # Show log info
    nmessage=nmessage,
    loss=objective_boost,
    training_options={'nepochs': maxiter,
                      'learning_rate': learning_rate,}
)
params_history.extend(np.array(partial_params_history))
loss_history.extend(np.array(partial_loss_history))
grads_history.extend(np.array(partial_grads_history))
fluctuations.extend(np.array(partial_fluctuations))

In [ ]:
import os
folder_path = f'results/vqe_training/{optimizer}_{nqubits}q_{nlayer}l_XXZ'
os.makedirs(folder_path, exist_ok=True)
path_param = folder_path + f'/vqe_params_hist_{nqubits}.npy'
path_loss = folder_path + f'/vqe_loss_hist_{nqubits}.npy'
# np.save(path_param, params_history)
# np.save(path_loss, loss_history)

## Step 2: load VQE results
We load the saved VQE training results from '.npy' files.

In [ ]:
# Training params must be provided if loading directly from file, 
# otherwise comment out the following 2 lines
optimizer = 'sgd'
nlayer = 1
# load from the results/vqe_training folder
import os
folder_path = f'results/vqe_training/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
os.makedirs(folder_path, exist_ok=True)
path_param = folder_path + f'vqe_params_hist_{nqubits}.npy'
path_loss = folder_path + f'vqe_loss_hist_{nqubits}.npy'

In [ ]:
vqe_params = np.load(path_param)
loss_history = np.load(path_loss)

In [ ]:
# plot the learning curve
plt. plot(loss_history)
plt.xlabel('Training epochs')
plt.ylabel('Loss function')
plt.title(f'VQE training with {nlayer} layer(s)')

We can compare the parameters:

In [ ]:
plt.plot(vqe_params[0], label='initial param')
plt.plot(vqe_params[-1], label='final param')
plt.legend()

We take the last set of parameters as the result of our VQE training and from there build our trained VQE circuit.

In [ ]:
vqe_param_final = vqe_params[-1]
ansatz_circ.set_parameters(vqe_param_final)
vqe_circ = ansatz_circ

## Step 2: GCI DBQA

### Double-bracket rotation

A double-bracket rotation is given by
$$
H_{k+1} = e^{s_kW_k}H_ke^{-s_kW_k}
$$
where $s$ is the rotation duration, the rotation generator $W_k:=[D_k, H_k]$, and $D_k$ is a diagonal operator. While it is possible to use a fixed $s$ and fixed $D$ for all iterations, variational strategies have been proposed [1] to improve the training efficiency. 

In this minimal example, we use the magnetic field parameterization for $D$, which considers a local magnetic field in the $z$-direction

$$
D_k(B^{(k)})=\sum_{j=1}^L \alpha_j^{(k)}Z_j,
$$

and we can use gradient descent to optimize the values of $\alpha_j^{(k)}$.

### Frame-shifted oracle
Since we have a warm-start VQE circuit $Q$, we address the problem of preparing the ground state of the input Hamiltonian $H_0$ by considering a frame-shifted $A_0 = Q^\dagger H_0 Q$. We would then have
$$
E^{(k)}=\langle \psi_k|H_0|\psi_k\rangle = \langle 0|A_{k+1}|0\rangle.
$$
Thus this warm-start mechanism allows us to interface VQE and DBQA by defining a common cost function.

### Group commutator iteration
In a group commutator iteration (GCI), we have $$J_{k+1}= U_k^\dagger J_k U_k$$
which is obtained by a product formula for $U_k$ [2].
We will use two examples
$$P_k = e^{is D_k} e^{is J_k} e^{-isD_k}$$
and
$$Q_k = e^{-is J_k}e^{is D_k} e^{is J_k} e^{-isD_k}$$

We can show that
$$J_{k+1}= P_k^\dagger J_k P_k= Q_k^\dagger J_k Q_k$$
because of a reduction by means of a commutator vanishing (the ordering was chosen on purpose).

This means that the group commutator $P_k$ and the reduced $Q_k$ schemes should give the same `GroupCommutatorIterationWithEvolutionOracles.h`. Additionally that should be also `DoubleBracketIteration.h` as long as the ordering is correct.

[1] L. Xiaoyue et al., “Strategies for optimizing double-bracket quantum algorithms,” Aug. 14, 2024, arXiv: arXiv:2408.07431. doi: 10.48550/arXiv.2408.07431. https://arxiv.org/abs/2408.07431 

[2] Y.-A. Chen, A. M. Childs, M. Hafezi, Z. Jiang, H. Kim, and Y. Xu, “Efficient product formulas for commutators and applications to quantum simulation,” Phys. Rev. Research, vol. 4, no. 1, p. 013191, Mar. 2022, doi: 10.1103/PhysRevResearch.4.013191. https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.4.013191 

### GCI procedure
We describe the procedure for preparing a GCI circuit:

1. Initialize base oracle evolution oracle of the input Hamiltonian: it outputs circuits that performs $e^{-itH_0}$; we use 2nd order Trotter-Suzuki and then recompile the 2 qubit unitaries into CNOT; evolution oracles are data structures to keep track of how the Hamiltonian simulation is done, it should output a circuit which will be used whenever DBQA needs to run the evolution governed by the input Hamiltonian $H_0$

2. Generate the frame-shifted evolution oracle: compose the VQE circuit $Q$ with the base oracle to get $A_0=Q^\dagger H_0 Q$ as the new Hamiltonian to train GCI on

3. Initialize the GCI: determine the group commutator approximation and define the parameterization for $D_k$. 

Using the magnetic field parameterization will use no CZ gates and will give qualitatively similar results. We recommend this in presence of noise, especially for the early steps when running more DBQA steps

In [ ]:
base_oracle = XXZ_EvolutionOracle.from_nqubits(
    nqubits=nqubits, delta=0.5, steps=2, order=2
)
# the following circuit performs exp{-0.01iH}
base_circ = base_oracle.circuit(t_duration=0.01)
print(base_circ.draw())

In [ ]:
frame_oracle = FrameShiftedEvolutionOracle.from_evolution_oracle(
    circuit_frame=vqe_circ,
    base_evolution_oracle=base_oracle,
)

In [ ]:
# db_rotation = DoubleBracketRotationType.group_commutator_third_order_reduced
dbr_type = DoubleBracketRotationType.group_commutator_reduced
gci = GroupCommutatorIterationWithEvolutionOracles(
        frame_oracle,
        dbr_type
    )

In [ ]:
eo_d_type = MagneticFieldEvolutionOracle
print(
        f"The gci mode is {gci.mode} rotation with {eo_d_type.__name__} as the oracle.\n"
    )

In [ ]:
# GCI settings
steps = 1
optimization_method = "sgd"
gd_epochs = 10
opt_options = {'gd_epochs':gd_epochs, }

In [ ]:
for gci_step_nmb in range(steps):
    logging.info(
        "\n################################################################################\n"
        + f"Optimizing GCI step {gci_step_nmb+1} with optimizer {optimization_method}"
        + "\n################################################################################\n"
    )
    it = time.time()
    if optimization_method == "sgd":
        params = (
            [4 - np.sin(x / 3) for x in range(nqubits)]
            if eo_d_type == MagneticFieldEvolutionOracle
            else [4 - np.sin(x / 3) for x in range(nqubits)] + nqubits * [1]
        )
        mode, best_s, best_b, eo_d = select_recursion_step_gd_circuit(
            gci,
            mode=dbr_type,
            eo_d_type=eo_d_type,
            params=params,
            step_grid=np.linspace(1e-5, 2e-2, 30),
            lr_range=(1e-3, 1),
            nmb_gd_epochs=gd_epochs,
            threshold=1e-4,
            max_eval_gd=30,
        )

        opt_dict = {"sgd_extras": "To be defined"}

    else:
        if gci_step_nmb == 0:
            p0 = [0.01]
            if eo_d_type == MagneticFieldEvolutionOracle:
                p0.extend([4 - np.sin(x / 3) for x in range(nqubits)])
            elif eo_d_type == IsingNNEvolutionOracle:
                p0.extend(
                    [4 - np.sin(x / 3) for x in range(nqubits)] + nqubits * [1]
                )

        else:
            p0 = [best_s]
            p0.extend(best_b)
        optimized_params, opt_dict = optimize_D(
            params=p0,
            gci=gci,
            eo_d_type=eo_d_type,
            mode=dbr_type,
            method=optimization_method,
            **opt_options,
        )
        best_s = optimized_params[0]
        best_b = optimized_params[1:]
        eo_d = eo_d_type.load(best_b)

    step_data = dict(
        best_s=best_s,
        eo_d_name=eo_d.__class__.__name__,
        eo_d_params=eo_d.params,
    )
    logging.info(f"Total optimization time required: {time.time() - it} seconds")
    gci.mode_double_bracket_rotation = dbr_type

    gci(best_s, eo_d, dbr_type)


### Result analysis

In this analysis, we will look at the following attributes:

1. Fidelity
$$
F(\rho, \sigma) = \text{tr}^{2}\left( \sqrt{\sqrt{\sigma}
        \rho^{\dagger}\sqrt{\sigma}} \right)
$$
in our case, we are comparing the approximated ground state of our hamiltonian $|\psi_k\rang=U_k|0\rang=V_k...V_0Q|0\rang$ and its analytical ground state. Note that when $L$ is odd, the ground states are degenerate, which can lead to wrong fidelity values.

2. Fidelity witness [1]
Fidelity witness is an experimentally-friendly measure to find the closeness between a target pure state $|\phi\rang$ and an experimental/simulated state $\rho$. It provides a lower bound for the fidelity.
Specifically,
$$ 1-|Tr(H_0\rho)-\lang\phi|H_0|\phi\rang|/\Delta \le F(|\phi\rang, \rho),
$$
where $\Delta$ is the first energy gap of the input Hamiltonian $H_0$.

[1] M. Gluza, M. Kliesch, J. Eisert, and L. Aolita, “Fidelity Witnesses for Fermionic Quantum Simulations,” Phys. Rev. Lett., vol. 120, no. 19, p. 190501, May 2018, doi: 10.1103/PhysRevLett.120.190501. https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.120.190501 

In [ ]:
from qibo.quantum_info.metrics import fidelity

In [ ]:
def report(vqe_circ, hamiltonian, gci, step, eo_d, mode):
    energies = hamiltonian.eigenvalues()
    ground_state_energy = float(energies[0])
    vqe_energy = float(hamiltonian.expectation(vqe_circ().state()))
    gci_loss = float(gci.loss(step, eo_d, mode))
    gap = float(energies[1] - energies[0])

    return (
        dict(
            nqubits=hamiltonian.nqubits,
            gci_loss=float(gci_loss),
            vqe_energy=float(vqe_energy),
            target_energy=ground_state_energy,
            diff_vqe_target=vqe_energy - ground_state_energy,
            diff_gci_target=gci_loss - ground_state_energy,
            gap=gap,
            diff_vqe_target_perc=abs(vqe_energy - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            diff_gci_target_perc=abs(gci_loss - ground_state_energy)
            / abs(ground_state_energy)
            * 100,
            fidelity_witness_vqe=1 - (vqe_energy - ground_state_energy) / gap,
            fidelity_witness_gci=1 - (gci_loss - ground_state_energy) / gap,
            fidelity_vqe=fidelity(vqe_circ().state(), hamiltonian.ground_state()),
            fidelity_gci=fidelity(
                gci.get_composed_circuit(best_s, eo_d, dbr_type)().state(), hamiltonian.ground_state()
            ),
        )
        | gci.get_gate_count_dict(gci.get_composed_circuit(best_s, eo_d, dbr_type))
    )
    
def print_report(report: dict):
    print(
        f"\
    The target energy is {report['target_energy']}\n\
    The VQE energy is {report['vqe_energy']} \n\
    The DBQA energy is {report['gci_loss']}. \n\
    The difference is for VQE is {report['diff_vqe_target']} \n\
    and for the DBQA {report['diff_gci_target']} \n\
    which can be compared to the spectral gap {report['gap']}.\n\
    The relative difference is \n\
        - for VQE {report['diff_vqe_target_perc']}% \n\
        - for DBQA {report['diff_gci_target_perc']}%.\n\
    The energetic fidelity witness of the ground state is: \n\
        - for the VQE  {report['fidelity_witness_vqe']} \n\
        - for DBQA {report['fidelity_witness_gci']}\n\
    The true fidelity is \n\
        - for the VQE  {report['fidelity_vqe']}\n\
        - for DBQA {report['fidelity_gci']}\n\
                    "
    )
    print(
        f"The boosting circuit used {report['nmb_cnot']} CNOT gates coming from compiled XXZ evolution and {report['nmb_cz']} CZ gates from VQE.\n\
For {report['nqubits']} qubits this gives n_CNOT/n_qubits = {report['nmb_cnot_relative']} and n_CZ/n_qubits = {report['nmb_cz_relative']}"
    )

In [ ]:
this_report = report(vqe_circ, ham, gci, best_s, eo_d, dbr_type)
print_report(this_report)

We see that based on the numerical simulation, a simple step of GCI DBQA further lowered the loss. From here, we can export the VQE+DBQA circuit in QASM format

In [ ]:
composed_circ = gci.get_composed_circuit(best_s, eo_d, dbr_type)

In [ ]:
vqe_c = models.Circuit.to_qasm(vqe_circ)
gci_c = models.Circuit.to_qasm(composed_circ)

In [ ]:
folder_path = f'results/circuit_qasm/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
os.makedirs(folder_path, exist_ok=True)
with open(folder_path + f"vqe_circ.qasm", "w") as file:
    file.write(vqe_c)

with open(folder_path + f"gci_circ.qasm", "w") as file:
    file.write(gci_c)